# Visualization of Reactome pathway enrichment results

In [ ]:
import numpy as np
import pandas as pd
import pathway_plot.visuals as pp
import plotly.express as px
from reactome2py import analysis
import os

In [ ]:
nb_dir = os.getcwd() 
data_dir = "../tests/"

# reactome pathway metadata
pathways_f = os.path.join(nb_dir, data_dir, "Pathway_List_In_Hierarchy_Release_79.txt")
pathways = pd.read_csv(pathways_f, sep='\t')

In [ ]:
# The current Reactome release version this notebook example results
!curl -X GET --header 'Accept: text/plain' 'https://reactome.org/ContentService/data/database/version'

Reactome pathway enrichment analysis results can often include hundreds of pathways from a short list of genes. To quickly summarize and help identify the most significant pathways in these results, a simple visualization is generated as shown below. We will start with example gene list available in the reactome analysis tools web application.

In [ ]:
pd.read_csv('../tests/reactome_example_gene_list.txt')

This example gene list can be submitted for pathway enrichment analysis generating the complete list of results below. These results were downloaded directly from the Reactome analysis service: 

In [ ]:
df = pd.read_csv('../tests/reactome_example_gene_list_enrichment_results.csv')
df

We see over 1000 pathways have been identified that contain at least one gene from our list. These results are exhaustive and will contain statistically insignificant results. We will filter based on FDR:

In [ ]:
df[df['Entities FDR'] < 0.01]

Now we have a much shorter list of pathways to work with but we still have tabular data. The fireworks and reacfoam visualizations would be available in the web analysis but we would like a simpler visualization for ranking the significance of our enrichment results. We want to see a simple visual summary of the enrichment results that can be used in typical analysis workflows and provide a quick summary of the most signifcant pathway results.

In [ ]:
pp.make_scatter_plot(pp.merge_results(df))

From the above visualization we can easily see the enrichment results and through plotly, we can use this graph interactively to examine potential pathways of interest.

## Scanpy use case

We will use the processed PBMC3k dataset as an example.

In [ ]:
import scanpy as sc
adata = sc.datasets.pbmc3k_processed()

In [ ]:
sc.pl.umap(adata, color='louvain')

Note the example dataset louvain groups were renamed to their celltype. Let's get the DEGs for each of these celltypes.

In [ ]:
sc.tl.rank_genes_groups(adata, groupby='louvain')
df = sc.get.rank_genes_groups_df(adata, group=None)

We don't want the results for every gene in the dataset. Let's filter out insignificant values and focus on genes with increased expression in CD4 T cells.

In [ ]:
df = df[df['group'] =='CD4 T cells']
df = df[df['pvals_adj'] < 1E-3]
df = df[df['logfoldchanges'] > 2]
df

In [ ]:
response = analysis.identifiers(','.join(df.names))
#response = identifiers_form(test_input_f)
token = response['summary']['token']
results = analysis.pathway2df(token)

# convert dtypes - results df returned with all string dtypes
convert = {
    "#Entities found": "int64",
    "#Entities total": "int64",
    "Entities ratio": "float64",
    "Entities pValue": "float64",
    "Entities FDR": "float64",
    "#Reactions found": "int64",
    "#Reactions total": "int64",
    "Reactions ratio": "float64"
}
results = results.astype(convert)

In [ ]:
results

In [ ]:
pp.make_scatter_plot(pp.merge_results(results))